# SHMTools Data Import Demo

This notebook demonstrates how to properly import and use the 3-story structure dataset with the corrected `import_3story_structure_shm()` function.

## Key Changes:
- Use `import_3story_structure_shm()` instead of the old `load_3story_data()`
- The new function returns a tuple: `(dataset, damage_states, state_list)`
- No more dictionary access - direct variable assignment

## Correct Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from examples.data import import_3story_structure_shm

# Load the 3-story structure dataset correctly
dataset, damage_states, state_list = import_3story_structure_shm()

# Convert state_list to expected format
states = state_list.flatten().astype(int)

print(f"Dataset shape: {dataset.shape}")
print(f"Damage states shape: {damage_states.shape}")  
print(f"State list shape: {state_list.shape}")
print(f"States (first 10): {states[:10]}")

# Set up default metadata (since old convenience functions are gone)
fs = 100  # Default sampling frequency
channels = ['Force', 'Channel 2', 'Channel 3', 'Channel 4', 'Channel 5']

print(f"Sampling frequency: {fs} Hz")
print(f"Channels: {channels}")

In [ ]:
# Check data availability - basic validation
print("Data validation:")
print(f"Dataset dimensions: {dataset.shape} (should be 8192, 5, 170)")
print(f"All finite values: {np.isfinite(dataset).all()}")
print(f"States range: {states.min()} to {states.max()}")
print(f"Unique states: {len(np.unique(states))}")

# Basic dataset statistics
print(f"\nDataset statistics:")
print(f"Mean: {np.mean(dataset):.6f}")
print(f"Std: {np.std(dataset):.6f}")
print(f"Min: {np.min(dataset):.6f}")
print(f"Max: {np.max(dataset):.6f}")

## Extract Channels for Analysis (Common Pattern)

In [ ]:
# Extract channels 2-5 (acceleration channels) as commonly done in examples
signals = dataset[:, 1:5, :]  # Skip channel 0 (force)
t, m, n = signals.shape

print(f"Extracted signals shape: {signals.shape}")
print(f"Time points (t): {t}")
print(f"Channels (m): {m} (channels 2-5)")  
print(f"Conditions (n): {n}")

# Create damage state labels (0=undamaged, 1=damaged)
# States 1-9 are undamaged, states 10-17 are damaged
binary_damage_states = (states >= 10).astype(int)

print(f"Undamaged instances: {np.sum(binary_damage_states == 0)}")
print(f"Damaged instances: {np.sum(binary_damage_states == 1)}")

In [ ]:
# Visualization of loaded data
plt.figure(figsize=(12, 8))

# Plot sample time histories
plt.subplot(2, 1, 1)
plt.plot(signals[:1000, 0, 0], 'k-', label='Channel 2, State 1 (Undamaged)', linewidth=0.8)
plt.plot(signals[:1000, 0, 90], 'r--', label='Channel 2, State 10 (Damaged)', linewidth=0.8)
plt.title('Sample Time Histories from Loaded Data')
plt.xlabel('Time points')
plt.ylabel('Acceleration')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot damage state distribution
plt.subplot(2, 1, 2)
unique_states, counts = np.unique(states, return_counts=True)
colors = ['blue' if s <= 9 else 'red' for s in unique_states]
plt.bar(unique_states, counts, color=colors, alpha=0.7)
plt.xlabel('Damage State')
plt.ylabel('Number of Tests')
plt.title('Distribution of Test Conditions by Damage State')
plt.xticks(unique_states)
plt.grid(True, alpha=0.3)

# Add legend
import matplotlib.patches as mpatches
undamaged_patch = mpatches.Patch(color='blue', alpha=0.7, label='Undamaged (1-9)')
damaged_patch = mpatches.Patch(color='red', alpha=0.7, label='Damaged (10-17)')
plt.legend(handles=[undamaged_patch, damaged_patch])

plt.tight_layout()
plt.show()

## Other Available Import Functions

In [ ]:
# Import other available datasets
from examples.data import (
    import_cbm_data_shm,
    import_active_sense1_shm, 
    import_sensor_diagnostic_shm,
    import_modal_osp_shm
)

# Test what datasets are available
datasets_info = {}

try:
    cbm_dataset, cbm_damage_states, cbm_state_list, cbm_fs = import_cbm_data_shm()
    datasets_info['CBM'] = f"Shape: {cbm_dataset.shape}, Fs: {cbm_fs} Hz"
except FileNotFoundError:
    datasets_info['CBM'] = "Not available (missing data_CBM.mat)"

try:
    sensor_healthy, sensor_example = import_sensor_diagnostic_shm()
    datasets_info['Sensor Diagnostic'] = f"Healthy: {sensor_healthy.shape}, Example: {sensor_example.shape}"
except FileNotFoundError:
    datasets_info['Sensor Diagnostic'] = "Not available (missing dataSensorDiagnostic.mat)"

try:
    nodes, elements, modes, resp_dof = import_modal_osp_shm()
    datasets_info['Modal OSP'] = f"Nodes: {nodes.shape}, Modes: {modes.shape}"
except FileNotFoundError:
    datasets_info['Modal OSP'] = "Not available (missing data_OSPExampleModal.mat)"

try:
    (waveform_base, waveform_test, sensor_layout, pair_list,
     border_struct, sample_rate, actuation_waveform, damage_location) = import_active_sense1_shm()
    datasets_info['Active Sensing'] = f"Base: {waveform_base.shape}, Test: {waveform_test.shape}"
except FileNotFoundError:
    datasets_info['Active Sensing'] = "Not available (missing data_example_ActiveSense.mat)"

print("Available datasets:")
for name, info in datasets_info.items():
    print(f"  {name}: {info}")

## Usage Pattern for Notebook Examples

In [ ]:
# Typical usage pattern for SHM analysis notebooks
print("Typical notebook setup pattern:")
print()

setup_code = '''
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from examples.data import import_3story_structure_shm

# Load data
dataset, damage_states, state_list = import_3story_structure_shm()
states = state_list.flatten().astype(int)

# Extract channels 2-5 for analysis  
data = dataset[:, 1:5, :]
t, m, n = data.shape

# Create binary damage labels
binary_labels = (states >= 10).astype(int)
'''

print(setup_code)

print("This pattern works for:")
print("- PCA outlier detection")
print("- SVD outlier detection") 
print("- Mahalanobis distance")
print("- Factor analysis")
print("- AR model analysis")
print("- And most other SHM examples")

## Summary

**Key changes from old system:**
1. Use `import_3story_structure_shm()` instead of `load_3story_data()`
2. Function returns tuple: `(dataset, damage_states, state_list)`
3. No more dictionary access - direct variable assignment
4. Convert `state_list` to flat array: `states = state_list.flatten().astype(int)`
5. Use default values for metadata (fs=100, channel names)

**This corrected pattern is now used in all fixed notebooks.**